In [1]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

from soma import aims
import numpy as np
import pandas as pd
import os

from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
a = ana.Anatomist()

existing QApplication: 0
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jc225751'


global modules:create qapp
 /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
done
Starting Anatomist.....
config file : /casa/home/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
loading module meshsplit
loading module palettecontrols
loading module volumepalettes
loading module save_resampled
loading module profilewindow
loading module statsplotwindow
loading module anacontrolmenu
loading module infowindow
loading module foldsplit
loading module simple_controls
loading module histogram
loading module valuesplotwindow
loading module ana_image_math
loading module modelGraphs
loading module paletteViewer
loading module bsa_proba
loading module gradientpalette
loading module selection
loading module gltf_io
all python modules loaded
Anatomist started.


: 

### Variable definitions

In [3]:
embeddings_file = "/neurospin/dico/data/deep_folding/current/models/Champollion_V0/FColl-SRh_left/09-45-57_0/imagen_random_embeddings/full_embeddings.csv"
participants_file = "/neurospin/dico/data/deep_folding/current/datasets/imagen/participants.csv"

In [4]:
label = "C0_L"

In [5]:
side = "L" # "R" or "L"
region = "F.Coll.-S.Rh."# "S.C.-sylv.", "ORBITAL", "CINGULATE", "SC-sylv", "F.I.P."
database='imagen'

# Building predictors

In [6]:
participants = pd.read_csv(participants_file, dtype={'Subject':object})
participants = participants.set_index("Subject")
participants = participants[[label]]

In [7]:
participants.head()

,C0_L
Subject,
000000001274,N
000000022453,N
000000075717,N
000000106601,N
000000106871,N


In [8]:
participants = participants[[label]].dropna()
participants = participants.replace({"N": 0, "?": 1, "Y": 2})
participants[label].unique()

array([0, 2, 1])

In [9]:
len(participants)

2008

In [10]:
embeddings = pd.read_csv(embeddings_file, dtype={'ID':object})
embeddings = embeddings.set_index("ID")

In [11]:
embeddings

,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10,...,dim247,dim248,dim249,dim250,dim251,dim252,dim253,dim254,dim255,dim256
ID,,,,,,,,,,,,,,,,,,,,,
000000001274,18.990326,-2.637387,-1.556609,24.083849,-4.040590,16.771635,1.599108,1.874225,-30.879646,7.682234,...,-39.123820,-5.678921,-21.181053,-8.374977,-22.606771,17.723831,22.112156,8.969368,1.264951,-8.303302
000000022453,12.898921,11.409929,-1.517000,11.232712,-11.452357,-22.173475,-13.694705,-76.292470,9.665255,-0.324796,...,-12.661349,36.823574,-15.200200,21.661432,-18.674618,-3.806372,-19.666197,13.631190,-9.625917,-19.115686
000000075717,30.708344,-16.544325,22.888418,13.623634,-15.397210,2.331861,-20.226252,-48.238945,8.221877,33.845337,...,-30.690780,23.065144,10.541023,11.809063,1.752206,27.995056,-0.763571,8.922271,-36.376415,-5.408909
000000106601,41.844320,-17.488672,7.799303,1.351549,-2.712715,19.838573,16.941830,-37.400078,-10.343578,-8.061617,...,-8.030893,-9.278765,14.555280,-32.189210,-10.937408,-60.053990,-43.740925,-7.268471,8.202720,3.963017
000000106871,8.364127,7.526300,-15.376553,3.306133,-11.545905,30.610796,18.608917,-15.162454,-18.026878,18.342451,...,-7.940546,-18.700014,-32.192627,-29.086016,0.391219,-40.444458,-51.270016,3.899933,15.671353,22.273834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
000099873252,-19.480698,8.121365,-0.632013,-1.488582,1.945429,15.073156,7.342053,8.937391,4.433087,14.693746,...,7.027314,15.098805,-11.002013,-29.663980,-4.432151,3.999415,-12.300156,6.636870,-1.525378,37.201977
000099875982,-8.174691,23.639969,9.456901,-10.451852,-15.690434,-7.064004,-27.314651,-1.260854,-8.963244,48.186417,...,-35.715330,5.931157,-7.287583,20.487270,2.193358,5.718078,-17.433664,-17.652185,-8.330072,-23.145802
000099888850,14.498680,-10.893403,-62.055540,-0.989860,8.192965,-9.461267,-2.491179,6.806447,-46.887814,-37.360107,...,47.196648,-15.416119,-25.187143,-3.479963,1.677261,-27.327866,16.858541,-17.493326,33.563816,-31.877012


In [12]:
merged = participants[[label]].merge(embeddings, left_index=True, right_index=True)

# Classifies
X = merged.drop([label], axis=1)
Y = merged[[label]].astype(float)

# Standard scaler
scaler = StandardScaler()
X[X.columns] = scaler.fit_transform(X)
df2 = X.copy()

# Makes OLS
df2 = sm.add_constant(df2)
model = sm.OLS(Y[label], df2)
results = model.fit()

p = results.params

# Gets predictor
df = Y.copy()
df["predicted"] = df2.dot(p)

results.fvalue

4.082092721737706

In [13]:
df.mean()

C0_L         0.47006
predicted    0.47006
dtype: float64

In [14]:
df.head()

,C0_L,predicted
000000001274,0.0,0.649837
000000022453,0.0,-0.116756
000000075717,0.0,-0.075189
000000106601,0.0,0.715403
000000106871,0.0,-0.152188


### Function definitions

In [15]:
def to_bucket(obj):
    if obj.type() == obj.BUCKET:
        return obj
    avol = a.toAimsObject(obj)
    c = aims.Converter(intype=avol, outtype=aims.BucketMap_VOID)
    abck = c(avol)
    bck = a.toAObject(abck)
    bck.releaseAppRef()
    return bck

def build_gradient(pal):
    gw = ana.cpp.GradientWidget(None, 'gradientwidget', pal.header()['palette_gradients'])
    gw.setHasAlpha(True)
    nc = pal.shape[0]
    rgbp = gw.fillGradient(nc, True)
    rgb = rgbp.data()
    npal = pal.np['v']
    pb = np.frombuffer(rgb, dtype=np.uint8).reshape((nc, 4))
    npal[:, 0, 0, 0, :] = pb
    npal[:, 0, 0, 0, :3] = npal[:, 0, 0, 0, :3][:, ::-1]  # BGRA -> RGBA
    pal.update()

def buckets_average(subject_id_list, dataset_name_list):
    dic_vol = {}
    dim = 0
    rep = 0
    if len(subject_id_list) == 0:
        return False
    while dim == 0 and rep < len(subject_id_list):
        if dataset_name_list[rep].lower() in ['ukb', 'ukbiobank', 'projected_ukb']:
            dataset = 'UkBioBank'
        elif dataset_name_list[rep].lower() in ['hcp']:
            dataset = 'hcp'
        elif dataset_name_list[rep].lower() in ['imagen']:
            dataset = 'imagen'
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"
        if os.path.isfile(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz'):
            sum_vol = aims.read(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz').astype(float)
            dim = sum_vol.shape
            sum_vol.fill(0)
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id_list[0]}_cropped_skeleton.nii.gz')
        rep += 1

    for subject_id, dataset in zip(subject_id_list,dataset_name_list):
        if dataset.lower() in ['ukb', 'ukbiobank',  'projected_ukb']:
            dataset = 'UkBioBank'
        elif dataset.lower() == 'hcp':
            dataset = 'hcp'
            
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"

        if os.path. isfile(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz'):
            vol = aims.read(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            # compare the dim with the first file dim

            if vol.np.shape != dim:
                raise ValueError(f"{subject_id_list[0]} and {subject_id} must have the same dim")

                
            # to have a binary 3D structure
            dic_vol[subject_id] = (vol.np > 0).astype(int)
            sum_vol.np[:] += (vol.np > 0).astype(int) 
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')

    sum_vol = sum_vol / len(subject_id_list)
    print(sum_vol, sum_vol.shape)
    return sum_vol

In [16]:
def visualize_averages_along_parameter(df, column_name, database, nb_subjects_per_average=200, nb_columns=3, nb_lines=1):
    # anatomist objects
    global _block
    global _average_dic
    global dic_packages
    nb_windows = nb_columns * nb_lines
    _average_dic = {}
    step = nb_subjects_per_average # number of subjects on which average is done
    # Creates the block if it has not been created
    try:
        _block
    except NameError:
        _block = a.createWindowsBlock(nb_columns)

    # Order according to column_name
    df = df.sort_values(column_name)
    predict_proba = df[[column_name]]

    # Creates dictionary of subjects to average
    dic_packages = {}
    for i in range(0,len(predict_proba),step):
        list_idx = (predict_proba.index[i:i+step].to_numpy())
        dic_packages[i//step] = list_idx
        
    # Ensures that last list contains the last step subjects
    list_idx = (predict_proba.index[-step:].to_numpy())
    dic_packages[i//step] = list_idx
        
    list_database = [database for i in range(step)]
    n_pack = len(dic_packages)

    # Loop of display averages
    list_pack = [int(np.ceil(i*n_pack/float(nb_windows))) for i in range(0,nb_windows)]
    list_pack[-1] = n_pack-1
    for i in list_pack:
        sum_vol = buckets_average(dic_packages[i], list_database)
        _average_dic[f'a_sum_vol{i}'] = a.toAObject(sum_vol)
        _average_dic[f'a_sum_vol{i}'].setPalette(minVal=0, absoluteMode=True)
        #wsum = a.createWindow('Sagittal', block=block)
        #wsum.addObjects(a_sum_vol)
        _average_dic[f'rvol{i}'] = a.fusionObjects(
            objects=[_average_dic[f'a_sum_vol{i}']],
            method='VolumeRenderingFusionMethod')
        _average_dic[f'rvol{i}'].releaseAppRef()
        # custom palette
        n = len(dic_packages[i])
        pal = a.createPalette('VR-palette')
        pal.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
        #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.5/n};0;1;1'
        build_gradient(pal)
        _average_dic[f'rvol{i}'].setPalette('VR-palette', minVal=0.05, absoluteMode=True)
        pal2 = a.createPalette('slice-palette')
        pal2.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
        #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.3/n};0;{0.7/n};1;1;1'
        build_gradient(pal2)
        _average_dic[f'a_sum_vol{i}'].setPalette('slice-palette')
        # rvol.palette().fill()
        _average_dic[f'wvr{i}'] = a.createWindow('3D', block=_block)
        _average_dic[f'wvr{i}'].addObjects(_average_dic[f'rvol{i}'])

### Visualization

In [17]:
# block = a.createWindowsBlock(10)
visualize_averages_along_parameter(df, "predicted", database)

<soma.aims.Volume_DOUBLE object at 0x70ee5abb8700> (29, 62, 36, 1)
Multitexturing present
function glActiveTexture found.
function glClientActiveTexture found.
function glBlendEquation found.
function glTexImage3D found.
function glMultiTexCoord3f found.
function glBindFramebuffer found.
function glBindRenderbuffer found.
function glFramebufferTexture2D found.
function glGenFramebuffers found.
function glGenRenderbuffers found.
function glFramebufferRenderbuffer found.
function glRenderbufferStorage found.
function glCheckFramebufferStatus found.
function glDeleteRenderbuffers found.
function glDeleteFramebuffers found.
Number of texture units: 4
function glUniform1f found.
function glUniform1i found.
function glUniform4fv found.
function glGetUniformLocation found.
function glMultTransposeMatrixf found.
function glAttachShader found.
function glDetachShader found.
function glCompileShader found.
function glCreateProgram found.
function glCreateShader found.
function glDeleteProgram fo

In [18]:
type(_block)

anatomist.direct.api.Anatomist.AWindowsBlock

In [19]:
_average_dic

{'a_sum_vol0': <anatomist.cpp.anatomist.SliceableObject object at 0x70ee85d783a0>,
 'rvol0': <anatomist.cpp.anatomist.MObject object at 0x70ee5abb8820>,
 'wvr0': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x70ee497b1000>,
 'a_sum_vol4': <anatomist.cpp.anatomist.SliceableObject object at 0x70ee4977a050>,
 'rvol4': <anatomist.cpp.anatomist.MObject object at 0x70ee4977a560>,
 'wvr4': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x70ee497cc8b0>,
 'a_sum_vol10': <anatomist.cpp.anatomist.SliceableObject object at 0x70ee4977a0e0>,
 'rvol10': <anatomist.cpp.anatomist.MObject object at 0x70ee497ccc10>,
 'wvr10': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x70ee497d4700>}

QLayout: Attempting to add QLayout "" to QWidget "", which already has a layout
